In [1]:
import pandas as pd
from datetime import datetime

# Estações

In [2]:
cimehgo = pd.read_excel(r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos Finais\CIMEHGO_FINAL.xlsx")
inmet = pd.read_excel(r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos Finais\INMET_FINAL.xlsx")

In [3]:
cimehgo_p = cimehgo.copy()
inmet_p = inmet.copy()

In [4]:
cimehgo_p = cimehgo_p.rename(columns={
    "ESTAÇÃO": "Estacao",
    "DATA/HORA(LOCAL)": "Data",
    "CHUVA(MM)": "Chuva_mm"
})

inmet_p = inmet_p.rename(columns={
    "ESTACAO": "Estacao",
    "Data": "Data",
    "Chuva(mm)": "Chuva_mm",                   
})
inmet_p["Chuva_mm"]   = pd.to_numeric(inmet_p["Chuva_mm"], errors="coerce").fillna(0)
cimehgo_p["Chuva_mm"] = pd.to_numeric(cimehgo_p["Chuva_mm"], errors="coerce").fillna(0)

In [5]:
inmet_p["Hora"] = pd.to_datetime(inmet_p["Hora"], format="%H:%M", errors="coerce").dt.time
inmet_p["Data"] = pd.to_datetime(inmet_p["Data"], errors="coerce")
inmet_p["Data"] = inmet_p.apply(lambda row: pd.Timestamp.combine(row["Data"], row["Hora"]), axis=1)

In [6]:
cimehgo_p = cimehgo_p[["Estacao", "Data", "Chuva_mm", "LATITUDE", "LONGITUDE"]]
inmet_p = inmet_p[["Estacao", "Data", "Chuva_mm", "LATITUDE", "LONGITUDE"]]

final_est = pd.concat([cimehgo_p, inmet_p], ignore_index=True)

In [7]:
final_est.to_csv(r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos para BI\DADOS_UNIFICADOS_ESTACOES.csv", index=False, encoding="latin1")

# Ocorrências

In [ ]:
lp = pd.read_excel(r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos Finais\Locais_por_Data_GEOCODED_FINAL.xlsx")
rp = pd.read_excel(r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos Finais\RELATOS_GEOCODED_FINAL.xlsx")
bp = pd.read_excel(r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos Finais\BOMBEIROS_GEOCODED_FINAL.xlsx")

In [9]:
locais = lp.copy()
relatos = rp.copy()
bombeiros = bp.copy()
bombeiros = bombeiros.rename(columns={"ENDEREÇO": "LOCAL"})

In [10]:
locais = locais[["DATA", "LOCAL", "Estação atribuída", "LATITUDE", "LONGITUDE"]]
relatos = relatos[["DATA", "LOCAL", "Estação atribuída", "LATITUDE", "LONGITUDE"]]
bombeiros = bombeiros[["DATA", "LOCAL", "Estação atribuída", "LATITUDE", "LONGITUDE"]]

In [11]:
locais["Fonte"] = "Noticias"
relatos["Fonte"] = "Relatos"
bombeiros["Fonte"] = "Bombeiros"

In [12]:
def parse_date(x):
    if pd.isna(x):
        return pd.NaT
    
    s = str(x).strip()

    # Remover dia da semana (ex.: "Sunday, January 7, 2024")
    if "," in s:
        partes = s.split(",", 1)  # remove só a 1ª vírgula
        if len(partes) == 2:
            s = partes[1].strip()  # vira "January 7, 2024"

    # Tentativa 1: conversão automática do pandas
    try:
        return pd.to_datetime(s, errors="raise", infer_datetime_format=True)
    except:
        pass

    # Tentativa 2: formato textual American ("April 7 2024" / "April 7, 2024")
    try:
        return datetime.strptime(s, "%B %d %Y")
    except:
        pass

    try:
        return datetime.strptime(s, "%B %d, %Y")
    except:
        pass

    # Tentativa final: retornar NaT
    return pd.NaT

In [13]:
locais["DATA"] = locais["DATA"].apply(parse_date)
relatos["DATA"] = relatos["DATA"].apply(parse_date)
bombeiros["DATA"] = bombeiros["DATA"].apply(parse_date)

C:\Users\thiag\AppData\Local\Temp\ipykernel_1804\3806628615.py:15: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(s, errors="raise", infer_datetime_format=True)
C:\Users\thiag\AppData\Local\Temp\ipykernel_1804\3806628615.py:15: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(s, errors="raise", infer_datetime_format=True)
C:\Users\thiag\AppData\Local\Temp\ipykernel_1804\3806628615.py:15: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is 

In [14]:
final_ocor = pd.concat([locais, relatos, bombeiros], ignore_index=True)

In [15]:
final_ocor

,DATA,LOCAL,Estação atribuída,LATITUDE,LONGITUDE,Fonte
0,2023-11-21,Setor Oeste,sem estação correspondente,-16.681220,-49.272167,Noticias
1,2023-11-21,T-63,sem estação correspondente,-16.686900,-49.264800,Noticias
2,2023-12-31,Vila Roriz,sem estação correspondente,-16.639066,-49.265592,Noticias
3,2024-01-07,Alameda Matisse,sem estação correspondente,-16.651585,-49.285933,Noticias
4,2024-01-07,Avenida Castelo Branco,Corrego Cascavel,-16.673351,-49.300192,Noticias
...,...,...,...,...,...,...
358,2020-11-20,RUA ANICUNS,Heliponto Goiania,-16.680882,-49.253269,Bombeiros
359,2024-11-19,RUA ANICUNS,Heliponto Goiania,-16.680882,-49.253269,Bombeiros
360,2024-10-27,RUA 5,Vila Brasília,-16.742280,-49.255600,Bombeiros
361,2025-01-12,RUA 21 DE ABRIL,sem estação correspondente,-16.622422,-49.218280,Bombeiros


In [16]:
final_ocor.to_excel(r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos para BI\DADOS_GEOCODED_UNIFICADOS.xlsx", index=False)

# Análise do Indicador 

In [17]:
# Para cada linha de final_ocor, veja se tem uma Estação atribuída que está dentro da lista de Estacao de final_est
valid_stations = final_est["Estacao"].unique()
final_ocor_filt = final_ocor[ final_ocor["Estação atribuída"].isin(valid_stations) ]

final_ocor_filt = final_ocor_filt.drop_duplicates()

In [18]:
df_alag = final_ocor.copy()
df_chuva = final_est.copy()

In [19]:
df_chuva['Data'] = pd.to_datetime(df_chuva['Data'])
df_chuva['data_dia'] = df_chuva['Data'].dt.date

df_alag['DATA'] = pd.to_datetime(df_alag['DATA'])
df_alag['data_dia'] = df_alag['DATA'].dt.date

df_alag = df_alag.rename(columns={'Estação atribuída': 'Estacao'})

In [20]:
estacoes_validas = set(df_chuva['Estacao'].unique())
df_alag_filtrado = df_alag[df_alag['Estacao'].isin(estacoes_validas)]

In [21]:
df_expandido = pd.merge(
    df_alag_filtrado,
    df_chuva,
    on=['Estacao', 'data_dia'],
    how='left'
)

In [22]:
estacoes_ausentes = (
    set(df_alag['Estacao'].unique())
    - set(df_chuva['Estacao'].unique())
)

In [23]:
# Tem no df_alag, mas nao no df_chuva
estacoes_ausentes

{'sem estação correspondente'}

In [35]:
df_expandido


,DATA,LOCAL,Estacao,LATITUDE_x,LONGITUDE_x,Fonte,data_dia,Data,Chuva_mm,LATITUDE_y,LONGITUDE_y
0,2024-01-07,Avenida Castelo Branco,Corrego Cascavel,-16.673351,-49.300192,Noticias,2024-01-07,NaT,NaN,NaN,NaN
1,2024-01-07,rua dom eduardo,Corrego Cascavel,-16.671880,-49.300995,Noticias,2024-01-07,NaT,NaN,NaN,NaN
2,2024-02-06,Avenida itaberai,Vila Jardim Pompeia,-16.615139,-49.252079,Noticias,2024-02-06,NaT,NaN,NaN,NaN
3,2024-02-06,Marginal Botafogo,Heliponto Goiania,-16.684613,-49.246549,Noticias,2024-02-06,NaT,NaN,NaN,NaN
4,2024-04-04,Alameda Angélico,Perim,-16.647395,-49.288590,Noticias,2024-04-04,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1175,2021-01-11,GO 020,Heliponto Goiania,-16.680882,-49.253269,Bombeiros,2021-01-11,2021-01-11 04:14:00,0.0,-16.680104,-49.250038
1176,2021-01-11,GO 020,Heliponto Goiania,-16.680882,-49.253269,Bombeiros,2021-01-11,2021-01-11 03:14:00,0.0,-16.680104,-49.250038
1177,2021-01-11,GO 020,Heliponto Goiania,-16.680882,-49.253269,Bombeiros,2021-01-11,2021-01-11 02:14:00,0.0,-16.680104,-49.250038
1178,2021-01-11,GO 020,Heliponto Goiania,-16.680882,-49.253269,Bombeiros,2021-01-11,2021-01-11 01:14:00,2.0,-16.680104,-49.250038


In [42]:
estatisticas_eventos = (
    df_expandido
    .groupby(['LOCAL', 'DATA', 'Estacao'])['Chuva_mm']
    .agg(['min','max','mean'])
    .reset_index()
)

In [ ]:
estatisticas_eventos

,LOCAL,DATA,Estacao,min,max,mean
0,ALAMEDA ALIANÇA,2025-03-11,Heliponto Goiania,NaN,NaN,NaN
1,ALAMEDA MARGINAL BOTAFOGO,2022-02-16,Heliponto Goiania,NaN,NaN,NaN
2,AV MARGINAL BOTAFOGO,2025-03-23,Heliponto Goiania,NaN,NaN,NaN
3,AV PADRE WENDEL,2020-01-21,Heliponto Goiania,NaN,NaN,NaN
4,AV. PARANAIBA,2021-01-11,Heliponto Goiania,0.0,2.0,0.091667
...,...,...,...,...,...,...
93,VIA ANTÔNIO CAMARGO,2021-12-20,Heliponto Goiania,0.0,27.8,1.258333
94,Vila Clemente,2024-04-04,Perim,NaN,NaN,NaN
95,Vila João Vaz,2024-10-27,Perim,0.0,10.6,0.995652
96,Vila Maria Rosa,2025-01-13,Vila Jardim Pompeia,0.0,0.0,0.000000


In [ ]:
df_chuva['data'] = pd.to_datetime(df_chuva['Data'])
df_alag['data'] = pd.to_datetime(df_alag['DATA'])

df_chuva['Data'] = pd.to_datetime(df_chuva['Data'])
df_alag['DATA'] = pd.to_datetime(df_alag['DATA'])

df_alag_ren = df_alag.rename(columns={
    'Estação atribuída': 'Estacao',
    'DATA': 'Data'
})

In [ ]:
#df_merge

NameError: name 'df_merge' is not defined

In [ ]:
"""df_merge = pd.merge(
    df_chuva,
    df_alag_ren[['Data', 'Estacao']], 
    on=['Estacao', 'Data'],
    how='inner'
)"""

In [85]:
data_u = final_ocor_filt

In [ ]:
# Para cada um desses pares de data e estação, ver no df final_est todas as ocorrencias com mesma data (vai precisar normalizar a hora) para pegar os valores de chuva (mm) do dia

In [ ]:
# Fazer três colunas, máxima, minima e média de chuva (mm) capturadas pelos pluviometros naquela data quando foi registrado um alagamento, com isso a gente pode estimar que
# caso tenha mais de uma ocorrência no mesmo local, os valores observados podem ser considerados indicadores de alagamento para a região da estação.

# No TCC isso precisa ser muito bem explicado para ficar obvio que os dados atuais são apenas estimativas e que um indicador de verdade precisa de muitos mais dados para ser formado